In [1]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from dxtb.config import ConfigCache
from dxtb import OutputHandler
from tblite.interface import Calculator


dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# LiH
# numbers = torch.tensor([3, 1], device=dd["device"])
# positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

numbers = torch.tensor([6, 6, 7, 7, 1, 1, 1, 1, 1, 1, 8, 8,], device=dd["device"])
positions = torch.tensor([
                [-3.81469488143921, +0.09993441402912, 0.00000000000000],
                [+3.81469488143921, -0.09993441402912, 0.00000000000000],
                [-2.66030049324036, -2.15898251533508, 0.00000000000000],
                [+2.66030049324036, +2.15898251533508, 0.00000000000000],
                [-0.73178529739380, -2.28237795829773, 0.00000000000000],
                [-5.89039325714111, -0.02589114569128, 0.00000000000000],
                [-3.71254944801331, -3.73605775833130, 0.00000000000000],
                [+3.71254944801331, +3.73605775833130, 0.00000000000000],
                [+0.73178529739380, +2.28237795829773, 0.00000000000000],
                [+5.89039325714111, +0.02589114569128, 0.00000000000000],
                [-2.74426102638245, +2.16115570068359, 0.00000000000000],
                [+2.74426102638245, -2.16115570068359, 0.00000000000000],
                ], **dd) # ** to use dd as kwargs

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
cache_config = ConfigCache(enabled=True, coefficients=True, mo_energies=True, density=True)
cache_config = ConfigCache(enabled=True, coefficients=True, mo_energies=True, density=True, overlap=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, CACHE_STORE_DENSITY=True, **dd)
calc.opts.cache = cache_config
OutputHandler.verbosity = 0

# instantiate a tblite calculator
calc_tblite = Calculator(
    method = "GFN1-xTB",
    numbers = numbers.numpy(),
    positions = positions.numpy(),
    uhf=None
)

calc_tblite.set("save-integrals", 1)
calc_tblite.set("verbosity", 0)
res = calc_tblite.singlepoint()

# Check if output dxtb and tblite are equal

In [2]:
from qcm_ml.features.xtb import _get_permutation_map_closedshell_convention, apply_perm_map

perm_map_dxtb = _get_permutation_map_closedshell_convention(numbers.numpy(), "dxtb")
perm_map_tblite = _get_permutation_map_closedshell_convention(numbers.numpy(), "tblite")

s_dxtb = apply_perm_map(calc.integrals.build_overlap(pos), perm_map_dxtb, positions.numpy())
s_tblite = apply_perm_map(res["overlap-matrix"], perm_map_tblite, positions.numpy())

h_dxtb = apply_perm_map(calc.integrals.build_hcore(pos), perm_map_dxtb, positions.numpy())
h_tblite = apply_perm_map(res["hamiltonian-matrix"], perm_map_tblite, positions.numpy())

assert torch.allclose(s_dxtb, torch.tensor(s_tblite, **dd), atol=1e-6)
assert torch.allclose(h_dxtb, torch.tensor(h_tblite, **dd), atol=1e-6)

# Check if 5 point stencil and manual jacobian are equal

In [3]:
from qcm_ml.features.xtb import get_jacobian, get_2body_grads

two_body_grads = get_2body_grads(numbers.numpy(), positions.numpy())

s_jac_dxtb = get_jacobian(s_dxtb, pos)
h_jac_dxtb = get_jacobian(h_dxtb, pos)

s_jac_tblite = two_body_grads[2]
h_jac_tblite = two_body_grads[3]

assert torch.allclose(s_jac_dxtb, torch.tensor(s_jac_tblite, **dd), atol=1e-6)
assert torch.allclose(h_jac_dxtb, torch.tensor(h_jac_tblite, **dd), atol=1e-6)

# Check if autograd.functional.jacobian and manual jacobian are equal

In [6]:
from dxtb.integrals.wrappers import hcore, overlap

def get_s(pos):
    return apply_perm_map(overlap(numbers, pos, dxtb.GFN1_XTB), perm_map_dxtb, positions.numpy())
def get_h(pos):
    return apply_perm_map(hcore(numbers, pos, dxtb.GFN1_XTB), perm_map_dxtb, positions.numpy())

s_jac_auto = torch.autograd.functional.jacobian(get_s, pos, strict=True)
h_jac_auto = torch.autograd.functional.jacobian(get_h, pos, strict=True)

assert torch.allclose(s_jac_auto, s_jac_dxtb, atol=1e-6)
assert torch.allclose(h_jac_auto, h_jac_dxtb, atol=1e-6)

In [75]:
def get_s(pos):
    return apply_perm_map(calc.integrals.build_overlap(pos), perm_map_dxtb, positions.numpy())
def get_h(pos):
    return apply_perm_map(calc.integrals.build_hcore(pos), perm_map_dxtb, positions.numpy())

# This does not work because it does not support the calc.integrals.build_*** method
# s_jac_auto = torch.autograd.functional.jacobian(get_s, pos, strict=True)
# h_jac_auto = torch.autograd.functional.jacobian(get_h, pos, strict=True)

# Also vectorized is not supported as custom einsum function is used.